In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from massspecgym.simulation_utils.spec_utils import get_ints_transform_func, get_ints_untransform_func, sparse_cosine_distance, batched_l1_normalize
from massspecgym.simulation_utils.misc_utils import safelog
from massspecgym.data.datasets import SimulationDataset
from massspecgym.transforms import SpecToMzsInts, MolToPyG, StandardMeta, MolToFingerprints

In [3]:
mzs = torch.arange(10)
ints = torch.rand_like(mzs,dtype=torch.float32)
ints = (ints / ints.max()) * 1000.
batch_idxs = torch.zeros_like(mzs)
print(mzs)
print(ints)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([ 146.7250,  949.8270, 1000.0000,  312.6480,  338.5695,  378.9861,
           9.3636,  216.3582,   75.6732,  404.0323])


In [4]:
ints_transform_func = get_ints_transform_func("log10t3")
ints_untransform_func = get_ints_untransform_func("log10t3")
ints_normalize_func = batched_l1_normalize 

In [5]:
ints_t = ints_transform_func(ints)
ints_ut = ints_untransform_func(ints_t)
print(ints)
print(ints_t)
print(ints_ut)

tensor([ 146.7250,  949.8270, 1000.0000,  312.6480,  338.5695,  378.9861,
           9.3636,  216.3582,   75.6732,  404.0323])
tensor([1.6982, 2.5019, 2.5242, 2.0221, 2.0564, 2.1049, 0.6150, 1.8640, 1.4187,
        2.1325])
tensor([ 146.7250,  949.8270, 1000.0000,  312.6481,  338.5695,  378.9861,
           9.3636,  216.3582,   75.6732,  404.0322])


In [15]:
log_probs = safelog(ints_normalize_func(ints, batch_idxs))
log_probs_t = safelog(ints_normalize_func(ints_transform_func(ints),batch_idxs))
log_probs_ut = safelog(ints_normalize_func(ints_untransform_func(log_probs_t.exp()),batch_idxs))
# log_probs_tut = safelog(ints_normalize_func(ints_transform_func(log_probs_ut.exp()),batch_idxs))
# log_probs_utut = safelog(ints_normalize_func(ints_untransform_func(log_probs_tut.exp()),batch_idxs))
print(log_probs.exp())
print(log_probs_t.exp())
print(log_probs_ut.exp())
# print(log_probs_tut.exp())
# print(log_probs_utut.exp())

tensor([0.0495, 0.1308, 0.0105, 0.0138, 0.0102, 0.0213, 0.2405, 0.0091, 0.0260,
        0.0111, 0.0514, 0.0097, 0.0304, 0.0133, 0.2762, 0.0124, 0.0086, 0.0340,
        0.0263, 0.0149])
tensor([0.0594, 0.0733, 0.0378, 0.0415, 0.0375, 0.0475, 0.0820, 0.0359, 0.0503,
        0.0385, 0.0599, 0.0367, 0.0525, 0.0410, 0.0840, 0.0401, 0.0351, 0.0540,
        0.0504, 0.0426])
tensor([0.0495, 0.1308, 0.0105, 0.0138, 0.0102, 0.0213, 0.2405, 0.0091, 0.0260,
        0.0111, 0.0514, 0.0097, 0.0304, 0.0133, 0.2762, 0.0124, 0.0086, 0.0340,
        0.0263, 0.0149])


In [7]:
tsv_pth = "../data/MassSpecGym_3.tsv"
meta_keys = ["adduct","precursor_mz","instrument_type","collision_energy"]
fp_types = ["morgan","maccs","rdkit"]
adducts = ["[M+H]+"]
instrument_types = ["QTOF","QFT","Orbitrap","ITFT"]
max_collision_energy = 200. # arbitrary
mz_from = 10.
mz_to = 1500.
spec_transform = SpecToMzsInts(
    mz_from=mz_from,
    mz_to=mz_to
)
mol_transform = MolToFingerprints(
    fp_types=fp_types
)
meta_transform = StandardMeta(
    adducts=adducts,
    instrument_types=instrument_types,
    max_collision_energy=max_collision_energy
)

ds = SimulationDataset(
    tsv_pth=tsv_pth,
    meta_keys=meta_keys,
    spec_transform=spec_transform,
    mol_transform=mol_transform,
    meta_transform=meta_transform,
    cache_feats=False)

/fs01/home/adamo/MassSpecGym/massspecgym/data/datasets.py:195: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  entry_df = pd.read_csv(self.tsv_pth, sep="\t")


In [8]:
item = ds[3]
mzs = item["spec_mzs"]
ints = (item["spec_ints"] / item["spec_ints"].max()) * 1000.
batch_idxs = torch.zeros_like(mzs,dtype=torch.int64)
log_probs = safelog(ints_normalize_func(ints, batch_idxs))
log_probs_t = safelog(ints_normalize_func(ints_transform_func(ints),batch_idxs))
log_probs_ut = safelog(ints_normalize_func(ints_untransform_func(log_probs_t.exp()),batch_idxs))
print(log_probs.exp())
print(log_probs_t.exp())
print(log_probs_t2.exp())
print(log_probs_ut.exp())

tensor([0.0495, 0.1308, 0.0105, 0.0138, 0.0102, 0.0213, 0.2405, 0.0091, 0.0260,
        0.0111, 0.0514, 0.0097, 0.0304, 0.0133, 0.2762, 0.0124, 0.0086, 0.0340,
        0.0263, 0.0149])
tensor([0.0594, 0.0733, 0.0378, 0.0415, 0.0375, 0.0475, 0.0820, 0.0359, 0.0503,
        0.0385, 0.0599, 0.0367, 0.0525, 0.0410, 0.0840, 0.0401, 0.0351, 0.0540,
        0.0504, 0.0426])
tensor([0.0495, 0.1308, 0.0105, 0.0138, 0.0102, 0.0213, 0.2405, 0.0091, 0.0260,
        0.0111, 0.0514, 0.0097, 0.0304, 0.0133, 0.2762, 0.0124, 0.0086, 0.0340,
        0.0263, 0.0149])


In [12]:
# from massspecgym.simulation_utils.misc_utils import LOG_ZERO, scatter_logsoftmax
# import numpy as np
# def log_ints_transform_func(log_y):
#     return 0.5 * log_y # sqrt
# def log_ints_untransform_func(log_y):
#     log_max_ints = 0.5*np.log(1000.)
#     log_y = log_y - torch.max(log_y) + log_max_ints 
#     log_y = 2. * log_y # square (inv sqrt)
#     log_y = torch.clamp(log_y, min=LOG_ZERO(log_y.dtype))
#     return log_y
# def log_ints_normalize_func(log_y, batch_idxs):
#     return scatter_logsoftmax(log_y, batch_idxs)

# item = ds[3]
# mzs = item["spec_mzs"]
# ints = (item["spec_ints"] / item["spec_ints"].max()) * 1000.
# batch_idxs = torch.zeros_like(mzs,dtype=torch.int64)
# log_probs = log_ints_normalize_func(safelog(ints), batch_idxs)
# log_probs_t = log_ints_normalize_func(log_ints_transform_func(safelog(ints)),batch_idxs)
# log_probs_ut = log_ints_normalize_func(log_ints_untransform_func(log_probs_t),batch_idxs)
# print(log_probs.exp())
# print(log_probs_t.exp())
# print(log_probs_ut.exp())

tensor([0.0495, 0.1308, 0.0105, 0.0138, 0.0102, 0.0213, 0.2405, 0.0091, 0.0260,
        0.0111, 0.0514, 0.0097, 0.0304, 0.0133, 0.2762, 0.0124, 0.0086, 0.0340,
        0.0263, 0.0149])
tensor([0.0599, 0.0973, 0.0276, 0.0316, 0.0272, 0.0393, 0.1320, 0.0257, 0.0434,
        0.0283, 0.0610, 0.0265, 0.0469, 0.0310, 0.1414, 0.0300, 0.0249, 0.0496,
        0.0436, 0.0329])
tensor([0.0495, 0.1308, 0.0105, 0.0138, 0.0102, 0.0213, 0.2405, 0.0091, 0.0260,
        0.0111, 0.0514, 0.0097, 0.0304, 0.0133, 0.2762, 0.0124, 0.0086, 0.0340,
        0.0263, 0.0149])
